In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import mstats
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
import keras_tuner as kt
#from keras_tuner import HyperParameters as hp

In [2]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
df = pd.read_csv('../../data/pre_train/aapl.csv')
df.columns = df.columns.str.lower()
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df = df.astype(float)

In [4]:
y = df['log_close']
X = df.drop('log_close', axis=1)

In [5]:
time_steps = 67
def create_sequences(X, y, time_steps=time_steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X.iloc[i:i + time_steps].values)
        y_seq.append(y.iloc[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

In [6]:
def evaluate_model(y_pred_train, y_pred_test, y_train, y_test):
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    
    print("MAE train:", mae_train)
    print("MAE test:", mae_test)
    print("RMSE train:", rmse_train)
    print("RMSE test:", rmse_test)
    print("R2 train:", r2_train)
    print("R2 test:", r2_test)

In [7]:
def plot_predictions(y_test_real, y_pred_test_real):
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_real, label="Real", color="blue")
    plt.plot(y_pred_test_real, label="Predicho", color="orange")
    plt.title("Predictions vs Real values")
    plt.legend()
    plt.show()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

In [10]:
X_train_seq, y_train_seq = create_sequences(pd.DataFrame(X_train_scaled), y_train)
X_test_seq, y_test_seq = create_sequences(pd.DataFrame(X_test_scaled), y_test)

In [11]:
apply_pca = True
n_components = 10

if apply_pca:
    pca = PCA(n_components=n_components)
    X_train_seq_flat = X_train_seq.reshape(-1, X_train_seq.shape[-1])
    X_test_seq_flat = X_test_seq.reshape(-1, X_test_seq.shape[-1])
    
    X_train_pca = pca.fit_transform(X_train_seq_flat)
    X_test_pca = pca.transform(X_test_seq_flat)

    X_train_seq = X_train_pca.reshape(-1, time_steps, n_components)
    X_test_seq = X_test_pca.reshape(-1, time_steps, n_components)

In [12]:
scaler_y = MinMaxScaler()
y_train_seq_scaled = scaler_y.fit_transform(y_train_seq.reshape(-1, 1)).flatten()
y_test_seq_scaled = scaler_y.transform(y_test_seq.reshape(-1, 1)).flatten()

In [ ]:
def build_model(hp):
    model = Sequential()
    
    model.add(LSTM(
        units=hp.Int('lstm_units', min_value=124, max_value=132, step=2),  
        return_sequences=False,
        input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
        kernel_regularizer=l2(0.0005)
    ))

    model.add(Dropout(0.2))

    model.add(Dense(
        64, activation='relu',
        kernel_regularizer=l2(0.001)
    ))

    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))

    learning_rate = hp.Float('learning_rate', min_value=0.007, max_value=0.013, step=0.002)
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
    
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='kt_fine_tuning',
    project_name='lstm_fine_tuning_precise'
)

In [22]:
tuner.search(X_train_seq, y_train_seq_scaled, 
             epochs=75, 
             batch_size=150,
             shuffle=False,
             validation_data=(X_test_seq, y_test_seq_scaled),
             callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

Trial 3 Complete [00h 02m 29s]
val_loss: 0.008864901028573513

Best val_loss So Far: 0.008516049012541771
Total elapsed time: 00h 08m 11s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
130               |132               |lstm_units
0.011             |0.007             |learning_rate

Epoch 1/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 452ms/step - loss: 0.4126 - mae: 0.5216 - val_loss: 0.1870 - val_mae: 0.2291
Epoch 2/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 242ms/step - loss: 0.2669 - mae: 0.3170 - val_loss: 0.1648 - val_mae: 0.2611
Epoch 3/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 222ms/step - loss: 0.1489 - mae: 0.1949 - val_loss: 0.1292 - val_mae: 0.1428
Epoch 4/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 202ms/step - loss: 0.1709 - mae: 0.2254 - val_loss: 0.1260 - val_mae: 0.1516
Epoch 5/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step - loss: 0.2337 - mae: 0.3097 - val_loss: 0.0934 - val_mae: 0.0613
Epoch 6/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - loss: 0.2530 - mae: 0.2974 - val_loss: 0.4550 -

KeyboardInterrupt: 

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Mejores hiperparámetros:", best_hps.values)

Best hps found: {'lstm_units': 128, 'l2': 0.001, 'dropout': 0.2, 'dense1_units': 64, 'l2_dense1': 0.001, 'dense2_units': 32, 'learning_rate': 0.01}


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [ ]:
evaluate_model(y_pred_train, y_pred_test, y_train_true, y_test_true)

MAE train: 0.06566370505226636
MAE test: 0.03390157973195015
RMSE train: 0.0801734531651841
RMSE test: 0.040072051088858404
R2 train: 0.8750941534398289
R2 test: 0.8711607671835455


In [ ]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import HyperParameters

# 🔹 Fijar la seed para asegurar reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 🔹 Función para construir el modelo
def build_model(hp):
    model = Sequential()
    model.add(LSTM(
        units=hp.Choice('lstm_units', values=[128]),
        return_sequences=False,
        input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
        kernel_regularizer=l2(hp.Choice('l2', values=[0.001]))  
    ))

    model.add(Dropout(hp.Choice('dropout', values=[0.2])))

    model.add(Dense(
        units=hp.Choice('dense1_units', values=[64]),  
        activation='relu',
        kernel_regularizer=l2(hp.Choice('l2_dense1', values=[0.001]))  
    ))

    model.add(Dense(
        units=hp.Choice('dense2_units', values=[32]),  
        activation='relu'
    ))

    model.add(Dense(1, activation='linear'))

    learning_rate = hp.Choice('learning_rate', values=[0.01])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
    
    return model

# 🔹 Configurar el tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='kt',
    project_name='lstm_finetuning'
)

# 🔹 Correr el *fine-tuning*
tuner.search(X_train_seq, y_train_seq_scaled, 
             epochs=75, batch_size=150,  
             shuffle=False,
             validation_data=(X_test_seq, y_test_seq_scaled),
             callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

# 🔹 Evaluar los *top 3* modelos
best_hps = tuner.get_best_hyperparameters(num_trials=3)
for i, hp in enumerate(best_hps):
    print(f"\n🔹 Evaluando el modelo {i+1}: {hp.values}")
    best_model = tuner.hypermodel.build(hp)
    history = best_model.fit(X_train_seq, y_train_seq_scaled, 
                             epochs=75, batch_size=150,
                             shuffle=False,
                             validation_data=(X_test_seq, y_test_seq_scaled),
                             callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])
    
    # Evaluar en test
    test_loss, test_mae = best_model.evaluate(X_test_seq, y_test_seq_scaled, verbose=0)
    print


Trial 1 Complete [00h 01m 18s]
val_loss: 0.0025765378959476948

Best val_loss So Far: 0.0025765378959476948
Total elapsed time: 00h 01m 18s

🔹 Evaluando el modelo 1: {'lstm_units': 128, 'l2': 0.001, 'dropout': 0.2, 'dense1_units': 64, 'l2_dense1': 0.001, 'dense2_units': 32, 'learning_rate': 0.01}
Epoch 1/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 217ms/step - loss: 0.3604 - mae: 0.4310 - val_loss: 0.1288 - val_mae: 0.1909
Epoch 2/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - loss: 1.2250 - mae: 0.8556 - val_loss: 0.1304 - val_mae: 0.2015
Epoch 3/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - loss: 0.4207 - mae: 0.4484 - val_loss: 0.1041 - val_mae: 0.0650
Epoch 4/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - loss: 0.1334 - mae: 0.1410 - val_loss: 0.1249 - val_mae: 0.1463
Epoch 5/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - loss: 0.1736 - mae: 0.2255 - val_loss: 0.1452 - val_mae: 0.2345
Epoch 6/75
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - loss: 0.1147 - mae: 0.1040 - val_loss: 0.2652 - val_mae: 0.2975
Epoch 

In [ ]:
y_pred_train_scaled = best_model.predict(X_train_seq)
y_pred_test_scaled = best_model.predict(X_test_seq)

y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled)
y_train_true = scaler_y.inverse_transform(y_train_seq_scaled.reshape(-1, 1))
y_test_true = scaler_y.inverse_transform(y_test_seq_scaled.reshape(-1, 1))

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [ ]:
evaluate_model(y_pred_train, y_pred_test, y_train_true, y_test_true)

MAE train: 0.0799315687024367
MAE test: 0.02619548564767189
RMSE train: 0.08913286844296825
RMSE test: 0.03196242439515628
R2 train: 0.8456177542571502
R2 test: 0.9180319859733228
